In [1]:
import numpy
import pandas as pd
import json
import sqlalchemy as db
from sqlalchemy.orm import Session
from sqlalchemy import text
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime

#I do not guarantee that the following code is 100% sql injection proof

<h2>Connect to (MySQL) Server</h2>

In [3]:
username = "root"
password = "password"
server = "localhost"
database = "quakes"

engine = db.create_engine(f"mysql+pymysql://{username}:{password}@{server}/{database}")
metadata = db.MetaData()

In [6]:
engine.table_names()

/var/folders/f2/jq5dp98n2pvcf43bjrz71s540000gn/T/ipykernel_13879/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Agencies',
 'Alerts',
 'Contributed',
 'Events',
 'IdMap',
 'MagnitudeTypes',
 'Status']

Create table variable for each table

In [7]:
events_tbl = db.Table('Events', metadata, autoload_with=engine)
idMap_tbl = db.Table('IdMap', metadata, autoload_with=engine)
agencies_tbl = db.Table('Agencies', metadata, autoload_with=engine)
status_tbl = db.Table('Status', metadata, autoload_with=engine)
alerts_tbl = db.Table('Alerts', metadata, autoload_with=engine)
contributed_tbl = db.Table('Contributed', metadata, autoload_with=engine)
mag_types_tbl = db.Table('MagnitudeTypes', metadata, autoload_with=engine)

<h2>Read data from files</h2>

In [8]:
def process_ids(local_id, local_ids, con):
    """
    Inserts (if not present) local_id + local_ids into IdMap table. 
    Throws an exception if:
        1. Some of local_id + local_ids are already assigned to different global ids, e.g. 
                local_ids contains 'us12345' and 'us67890' with 'us12345' --> 1 and 'us67890' --> 2.
        2. There was an event with the same local_id.
        
        Parameters:
            local_id: int --- primary id associated with the event.
            local_ids: Iterable[int] --- other ids associated with the event (possibly including local_id).
            con: sqlalchemy.engine.Connection --- connection to sql server.
            
        Output: 
            (primary_key_to_record_with_local_id: int, global_id_assigned_to_local_id: int).
    """
    
    #pk(IdMap) for local_id (or None if it wasn't inserted)
    res_id = con.execute(db.select(idMap_tbl.c.id).where(idMap_tbl.c.local_id == local_id)).scalar()
    
    #throw exception if there was an event with the same local_id
    if res_id is not None and \
       con.execute(db.select(events_tbl.c.id).where(events_tbl.c.local_id == res_id)).rowcount > 0:
        raise Exception(f"Duplicate local_id {local_id}.")
    
    #local_ids = set of local_id + local_ids
    local_ids = set(local_ids)
    local_ids.add(local_id)
    
    #pk(IdMap)'s for all local_ids
    res = con.execute(db.select(idMap_tbl.c.id).where(idMap_tbl.c.local_id.in_(local_ids)))
    
    #if all local_ids were already assigned some global_id, fetch that global_id.
    ###possibly removable
    if res.rowcount == len(local_ids):
        stmt = db.select(idMap_tbl.c.id, idMap_tbl.c.global_id).where(idMap_tbl.c.local_id == local_id)
        res2 = list(con.execute(stmt))
        return res2[0][0], res2[0][1]
    
    #find all global ids corresponding to local ids
    globals = list(con.execute(db.select(idMap_tbl.c.global_id).where(idMap_tbl.c.local_id.in_(local_ids)).distinct()))
    
    #throw if local ids are assigned more than one global id
    if len(globals) > 1:
        raise Exception('Overlapping is not supported.')
    
    
    new_global = None
    #if some global id was assigned, get it
    if (len(globals) > 0):
        new_global = globals[0][0]
    #otherwise define new global id as max of all global ids + 1 (if table is empty return 1)
    else:
        possible_id = con.execute(text("SELECT MAX(global_id) FROM IdMap")).scalar()
        new_global = 1 if possible_id is None else possible_id + 1

    #insert those local ids into IdMap table that are not already present
    for lid in local_ids:
        insert_stmt = db.dialects.mysql \
                                 .insert(idMap_tbl).values(local_id=lid, global_id=new_global)
        res = con.execute(insert_stmt.on_duplicate_key_update(global_id=insert_stmt.inserted.global_id))   
    
    #find pk(IdMap) for local_id
    db.select(idMap_tbl.c.id).where(idMap_tbl.c.local_id == local_id)
    id = con.execute(db.select(idMap_tbl.c.id).where(idMap_tbl.c.local_id == local_id)).scalar()
    return id, new_global


In [9]:
def process_sources(sources, event_global_id, con):
    """
    Updates Contributed table.
    Throws an exception if one of sources is not already present in Agencies table.
    
        Parameters:
            sources: Iterable[str] --- agencies that contributed to the event
            event_global_id: int --- primary key to the event in question
            con: sqlalchemy.engine.Connection --- connection to sql server
        
        Output:
            No output
    """
    
    sources = set(sources)

    #find pk(Agencies)'s for all sources
    stmt = db.select(agencies_tbl.c.id).where(agencies_tbl.c.abbreviation.in_(sources))
    res = con.execute(stmt)
    
    #throw if some agency is not found in Agencies table
    if res.rowcount != len(sources):
        raise Exception(f'Some agency among {sources} is not found. Add all new agencies to Agencies table.')
    
    #update Contributed table
    select_stmt = db.select(agencies_tbl.c.id, event_global_id).where(agencies_tbl.c.abbreviation.in_(sources))
    insert_stmt = db.dialects.mysql.insert(contributed_tbl) \
                            .from_select(['agency', 'event_global_id'], select_stmt)
    con.execute(insert_stmt.on_duplicate_key_update(agency=insert_stmt.inserted.agency))

In [10]:
def process_geojson_entry(entry, con, debug):
    """
    Processes an event (entry) into schema.
    Throws if agency(network/net) is null.
    
        Parameters:
            entry: dict --- geojson for the event
            con: sqlalchemy.engine.Connection --- connection to sql server
        
        Output:
            No output
    """
    #do nothing if entry is not an earthquake
    if entry['properties']['type'] != 'earthquake':
        warnings.warn("Not an earthquake.")
        return
    
    local_id = entry['id']
    [long, lat, depth] = entry['geometry']['coordinates']
    props = entry['properties']
    
    #process ids
    id_IdMap, global_id = process_ids(local_id, filter(None, props['ids'].split(',')), con)
    
    #process alert, status and magnitude type
    alert_id = con.execute(db.select(alerts_tbl.c.id).where(alerts_tbl.c.name == props['alert'])).scalar()
    status_id = con.execute(db.select(status_tbl.c.id).where(status_tbl.c.name == props['status'])).scalar()
    mag_type_id = con.execute(db.select(mag_types_tbl.c.id).where(mag_types_tbl.c.name == props['magType'])).scalar()
    
    #find pk(Agencies) for agency
    net_id = con.execute(db.select(agencies_tbl.c.id).where(agencies_tbl.c.abbreviation == props['net'])).scalar()
    if net_id is None:
        raise Exception('Agency is required.')
    
    #process sources
    sources = set(filter(None, props['sources'].split(',')))
    sources.add(props['net'])
    process_sources(sources, global_id, con)
    
    #insert event into Events table
    con.execute(events_tbl.insert().values(
                      magnitude=props['mag'], place=props['place'], \
                      time=datetime.datetime.fromtimestamp(props['time']//1000), \
                      updated=datetime.datetime.fromtimestamp(props['updated']//1000), \
                      timezone_offset=props['tz'], url=props['url'], \
                      detail=props['detail'], felt=props['felt'], cdi=props['cdi'], \
                      mmi=props['mmi'], alert=alert_id, status=status_id, \
                      tsunami=props['tsunami'], significance=props['sig'], network=net_id, \
                      n_stations=props['nst'], dmin=props['dmin'], rms=props['rms'], \
                      gap=props['gap'], magnitude_type=mag_type_id, title=props['title'], \
                      longitude=long, latitude=lat, depth=depth, \
                      local_id=id_IdMap, types=props['types'], code=props['code'] \
    ))

In [11]:
def process_geojson_file(geojson, con):
    """
    Processes a geojson file into schema 
    
        Parameters:
            geojson: dict --- geojson with events
            con: sqlalchemy.engine.Connection --- connection to sql server
        
        Output:
            No output
    """
    j = 1
    records = geojson['features']
    for record in records:
        process_geojson_entry(record, con)
        
        if j % 100 == 0:
            print(f"{j} entries processed.")
        j = j + 1

<h4>Geojson files:</h4>

In [12]:
all_files = !ls
all_files = list(filter(lambda x: x.startswith("quakes-"), all_files))
all_files

['quakes-00000-20000.json',
 'quakes-20000-40000.json',
 'quakes-40000-60000.json',
 'quakes-60000-80000.json',
 'quakes-80000-100000.json']

<h4>Read them inside the server</h4>

In [13]:
process_geojson_entry.j = 1
with Session(engine) as session:
    for file in all_files:
        with open(file) as f:
            session.begin()
            data = json.load(f)
            process_geojson_file(data, session)
            session.commit()

100 entries processed.
200 entries processed.
300 entries processed.
400 entries processed.
500 entries processed.
600 entries processed.
700 entries processed.
800 entries processed.
900 entries processed.
1000 entries processed.
1100 entries processed.
1200 entries processed.
1300 entries processed.
1400 entries processed.
1500 entries processed.
1600 entries processed.
1700 entries processed.
1800 entries processed.
1900 entries processed.


/var/folders/f2/jq5dp98n2pvcf43bjrz71s540000gn/T/ipykernel_13879/3494826104.py:15: UserWarning: Not an earthquake.
  warnings.warn("Not an earthquake.")


2000 entries processed.
2100 entries processed.
2200 entries processed.
2300 entries processed.
2400 entries processed.
2500 entries processed.
2600 entries processed.
2700 entries processed.
2800 entries processed.
2900 entries processed.
3000 entries processed.
3100 entries processed.
3200 entries processed.
3300 entries processed.
3400 entries processed.
3500 entries processed.
3600 entries processed.
3700 entries processed.
3800 entries processed.
3900 entries processed.
4000 entries processed.
4100 entries processed.
4200 entries processed.
4300 entries processed.
4400 entries processed.
4500 entries processed.
4600 entries processed.
4700 entries processed.
4800 entries processed.
4900 entries processed.
5000 entries processed.
5100 entries processed.
5200 entries processed.
5300 entries processed.
5400 entries processed.
5500 entries processed.
5600 entries processed.
5700 entries processed.
5800 entries processed.
5900 entries processed.
6000 entries processed.
6100 entries pro

35100 entries processed.
35200 entries processed.
35300 entries processed.
35400 entries processed.
35500 entries processed.
35600 entries processed.
35700 entries processed.
35800 entries processed.
35900 entries processed.
36000 entries processed.
36100 entries processed.
36200 entries processed.
36300 entries processed.
36400 entries processed.
36500 entries processed.
36600 entries processed.
36700 entries processed.
36800 entries processed.
36900 entries processed.
37000 entries processed.
37100 entries processed.
37200 entries processed.
37300 entries processed.
37400 entries processed.
37500 entries processed.
37600 entries processed.
37700 entries processed.
37800 entries processed.
37900 entries processed.
38000 entries processed.
38100 entries processed.
38200 entries processed.
38300 entries processed.
38400 entries processed.
38500 entries processed.
38600 entries processed.
38700 entries processed.
38800 entries processed.
38900 entries processed.
39000 entries processed.


67900 entries processed.
68000 entries processed.
68100 entries processed.
68200 entries processed.
68300 entries processed.
68400 entries processed.
68500 entries processed.
68600 entries processed.
68700 entries processed.
68800 entries processed.
68900 entries processed.
69000 entries processed.
69100 entries processed.
69200 entries processed.
69300 entries processed.
69400 entries processed.
69500 entries processed.
69600 entries processed.
69700 entries processed.
69800 entries processed.
69900 entries processed.
70000 entries processed.
70100 entries processed.
70200 entries processed.
70300 entries processed.
70400 entries processed.
70500 entries processed.
70600 entries processed.
70700 entries processed.
70800 entries processed.
70900 entries processed.
71000 entries processed.
71100 entries processed.
71200 entries processed.
71300 entries processed.
71400 entries processed.
71500 entries processed.
71600 entries processed.
71700 entries processed.
71800 entries processed.
